# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) which are [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/deep-learning-v2-pytorch/intro-to-pytorch'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os

os.chdir('./gdrive/My Drive/deep-learning-v2-pytorch/intro-to-pytorch')
!pwd

/content/gdrive/My Drive/deep-learning-v2-pytorch/intro-to-pytorch


In [0]:
cat_dogs = 'https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip'
#!wget $cat_dogs

In [0]:
#!unzip -q Cat_Dog_data.zip

In [5]:
!ls

 Cat_Dog_data
 Cat_Dog_data.zip
'Part 8 - Transfer Learning (Exercises)_aw.ipynb'
'Part 8 - Transfer Learning (Exercises).ipynb'
'Part 8 - Transfer Learning (Solution).ipynb'


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [0]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, 
                                                             std=std)])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=mean, 
                                                            std=std)])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [0]:
model = models.densenet121(pretrained=True)
#model

This model is built out of two main parts, the **features** and the **classifier**.

The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer at the end, `(classifier): Linear(in_features=1024, out_features=1000)`. 

This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the feature layers will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

So our job is to build our own feature classifier that classifies into two labels: cats or dogs.

In [0]:
# Freeze parameters for the feature detectors so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Build a new classifier from an OrderedDict
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                                        ('fc1', nn.Linear(1024, 500)),
                                        ('relu', nn.ReLU()),
                                        ('fc2', nn.Linear(500, 2)),
                                        ('output', nn.LogSoftmax(dim=1))
                                        ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [0]:
import time

In [28]:
for device in ['cpu', 'cuda']:
    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        start = time.time()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

KeyboardInterrupt: ignored

You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet (it's also a good model to try out first). Make sure you are **only training the classifier and the parameters for the features part are frozen**.

In [0]:
#model.parameters

In [0]:
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    
    for images, labels in testloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class, compare with true label
        equality = (labels.data == ps.max(1)[1])
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss, accuracy


def train(model, trainloader, testloader, criterion, optimizer, epochs=5, print_every=40):
    steps = 0
    running_loss = 0
    in_dims = 244 * 244 * 3
    
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            steps += 1

            # Flatten images
            images.resize_(images.size()[0], in_dims)

            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                # Model in inference mode, dropout is off
                model.eval()

                # Turn off gradients for validation, will speed up inference
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

                running_loss = 0

                # Make sure dropout and grads are on for training
                model.train()

In [38]:
## TODO: Use a pretrained model to classify the cat and dog images
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using device', device)

criterion = nn.NLLLoss()
    
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.to(device)

epochs = 2
running_loss = 0
print_every = 100

for e in range(epochs):
    start = time.time()
    
    for ii, (inputs, labels) in enumerate(trainloader):
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
        if ii % print_every == 0:
            # Model in inference mode, dropout is off
            model.eval()
            # Turn off gradients for validation, will speed up inference
            with torch.no_grad():
                test_loss, accuracy = validation(model, testloader, criterion)

            print("Epoch: {}/{}.. ".format(e, epochs),
                  "ii: {}.. ".format(ii),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

            running_loss = 0
            # Make sure dropout and grads are on for training
            model.train()

    print(f"Device = {device}; Time per epoch: {(time.time() - start):.3f} seconds")

using device cuda:0
Epoch: 0/2..  ii: 0..  Training Loss: 0.003..  Test Loss: 0.054..  Test Accuracy: 0.982
Epoch: 0/2..  ii: 100..  Training Loss: 0.176..  Test Loss: 0.083..  Test Accuracy: 0.970
Epoch: 0/2..  ii: 200..  Training Loss: 0.146..  Test Loss: 0.045..  Test Accuracy: 0.982
Epoch: 0/2..  ii: 300..  Training Loss: 0.162..  Test Loss: 0.044..  Test Accuracy: 0.984
Device = cuda:0; Time per epoch: 0.071 seconds
Epoch: 1/2..  ii: 0..  Training Loss: 0.074..  Test Loss: 0.041..  Test Accuracy: 0.986
Epoch: 1/2..  ii: 100..  Training Loss: 0.143..  Test Loss: 0.036..  Test Accuracy: 0.987
Epoch: 1/2..  ii: 200..  Training Loss: 0.159..  Test Loss: 0.044..  Test Accuracy: 0.982
Epoch: 1/2..  ii: 300..  Training Loss: 0.138..  Test Loss: 0.038..  Test Accuracy: 0.985
Device = cuda:0; Time per epoch: 0.070 seconds
